In [1]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set download directory
download_dir = "/content/drive/MyDrive/Kaggle"

# Create the directory if it doesn't exist
os.makedirs(download_dir, exist_ok=True)


# Upload kaggle.json (if not already uploaded)
from google.colab import files

# Option 1: Upload kaggle.json directly in Colab
# Uncomment the next line if you haven't uploaded the kaggle.json yet
# uploaded = files.upload()

# Configure Kaggle
os.makedirs('/root/.kaggle', exist_ok=True)
!cp /content/drive/MyDrive/Kaggle/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# Import and authenticate Kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()



Mounted at /content/drive


In [38]:

api.dataset_download_files('khushikhushikhushi/dog-breed-image-dataset', path=download_dir, unzip=True)

print(f"Downloaded files to: {download_dir}")

2024-11-28 00:17:02,612 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /api/v1/datasets/download/khushikhushikhushi/dog-breed-image-dataset?datasetVersionNumber=None


Dataset URL: https://www.kaggle.com/datasets/khushikhushikhushi/dog-breed-image-dataset
Downloaded files to: /content/drive/MyDrive/Kaggle/dogs


In [39]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [40]:
def load_images_from_folder(folder, image_size=(150, 150)):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
                img = Image.open(img_path)
                img = img.resize(image_size)
                img = np.array(img)
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

In [41]:
data_dir = '/content/drive/MyDrive/Kaggle/dogs/dataset'
image_size = (150, 150)

In [57]:
# prompt: get folder names into list in data_dir

import os

data_dir = '/content/drive/MyDrive/Kaggle/dogs/dataset'
folder_names = [f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))]
folder_names

['Beagle',
 'Boxer',
 'Bulldog',
 'Dachshund',
 'German_Shepherd',
 'Golden_Retriever',
 'Labrador_Retriever',
 'Poodle',
 'Rottweiler',
 'Yorkshire_Terrier']

In [42]:
X, y = load_images_from_folder(data_dir, image_size)

In [43]:
X = X / 255.0

In [44]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [45]:
# Convert labels to one-hot encoding
y = to_categorical(y)

In [56]:
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [46]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes to confirm
print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_val shape: {y_val.shape}')

X_train shape: (773, 150, 150, 3)
X_val shape: (194, 150, 150, 3)
y_train shape: (773, 10)
y_val shape: (194, 10)


In [47]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.summary()

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_35 (Conv2D)                   │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_35 (MaxPooling2D)      │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_36 (Conv2D)                   │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_36 (MaxPooling2D)      │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_37 (Conv2D)                   │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_37 (MaxPooling2D)      │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,038,794 (72.63 MB)

 Trainable params: 19,038,794 (72.63 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val)
)

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 15s 283ms/step - accuracy: 0.1310 - loss: 2.7965 - val_accuracy: 0.1856 - val_loss: 2.1503
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.3950 - loss: 1.8024 - val_accuracy: 0.8093 - val_loss: 0.8018
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.8390 - loss: 0.5767 - val_accuracy: 0.9639 - val_loss: 0.2521
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.9676 - loss: 0.1305 - val_accuracy: 0.9639 - val_loss: 0.1463
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9851 - loss: 0.0698 - val_accuracy: 0.9897 - val_loss: 0.1019
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9862 - loss: 0.0551 - val_accuracy: 0.9897 - val_loss: 0.1270
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9872 - loss: 0.0628 - val_accuracy: 0.9897 - val_loss: 0.1258
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9956 - loss: 0.0168 - val_accuracy: 0.9897 -

In [50]:
import tensorflow as tf

# Save the model in TensorFlow SavedModel format
tf.saved_model.save(
    model,  # The model you want to save
    export_dir='/content/drive/MyDrive/Kaggle/dog_model'  # Directory to save the model
)


In [51]:
!pip install coremltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00


In [58]:
import coremltools as ct

mlmodel = ct.convert(
    '/content/drive/MyDrive/Kaggle/dog_model',  # Path to the SavedModel directory
    source="tensorflow",
    inputs=[ct.ImageType(shape=(1, 150, 150, 3))],  # Specify input shape
    classifier_config=ct.ClassifierConfig(class_labels=folder_names)
)




Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 88.52 passes/s]


In [59]:
mlmodel.save('/content/drive/MyDrive/Kaggle/dogs/DogBreedClassifier.mlpackage')